# Import Package

In [1]:
#dataframe
import pandas as pd

#computation
import numpy as np


#NLP lib
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


# string operations
import string 
import re

# general imports
import math


#visualization
import matplotlib.pyplot as plt

#others
import collections
from collections import Counter
import itertools
from itertools import islice
from itertools import chain
import pickle

#pre-build package
from utils import *
from Invertedfile_TFIDF import Invertedfile_TFIDF

#auto reload
%load_ext autoreload
%autoreload 2

# Load Data

In [2]:
df = pd.read_csv('data.tsv', sep='\t',error_bad_lines=False)
df.head()

/opt/anaconda3/envs/nlp_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 83032: expected 6 fields, saw 7\n'
b'Skipping line 154657: expected 6 fields, saw 7\n'
b'Skipping line 323916: expected 6 fields, saw 7\n'
/opt/anaconda3/envs/nlp_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,qid1,qid2,question1,question2,is_duplicate
0,133273,213221,213222.0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
2,360472,364011,490273.0,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0
4,183004,279958,279959.0,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0


In [3]:
df.dropna(inplace=True)
#drop rows with dupplication
df = df.drop_duplicates(subset = ['question1'], keep = 'last')
df = df.drop_duplicates(subset = ['question2'], keep = 'last')
df.nunique()

id              232203
qid1            232203
qid2            232202
question1       232203
question2       232203
is_duplicate         2
dtype: int64

### Test Set

In [4]:
df.reset_index(inplace=True)

In [5]:
#Get the first 100 "is duplicate" rows for testing
df_test = df[df['is_duplicate'] == 1].iloc[:100]
df_test.head()

,index,id,qid1,qid2,question1,question2,is_duplicate
1,1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
6,13,11568,22332,22333.0,Which is the best book to study TENSOR for gen...,Which is the best book for tensor calculus?,1.0
11,24,33995,62359,62360.0,How does an IQ test work and what is determine...,How does IQ test works?,1.0
12,25,244506,357159,357160.0,Is it safe to use Xiaomi mobile phones?,Is it safe or unsafe to use Xiaomi Products?,1.0
14,27,375073,506056,506057.0,What are the best books on cosmology?,Which is the best book for cosmology?,1.0


# Text Preprocessing

In [6]:
df['processed_ques2'] = df.question2.apply(lambda x: text_preprocessing(x))

# TFIDF with Inverted File

In [7]:
model1 = Invertedfile_TFIDF(df[:5000])

Build vocabulary...
Build TF-IDF matrix...
Build inverted file...
Model is ready to query!


In [8]:
top2  = model1.accuracy(df_test, 2) 
top5  = model1.accuracy(df_test, 5) 

print("Accuracy with k = 2: ", top2)
print("Accuracy wit k = 5: ", top5)

Accuracy with k = 2:  0.9
Accuracy wit k = 5:  0.97


In [9]:
model1.query_engine()

Your query is: level
 What is the difference between user level and kernel level threads? - 2.6552614282206957
-----------------------------------------------------
 Why all the semiconductor have same Fermi level? - 2.6552614282206957
-----------------------------------------------------
 What is the purpose of philosophy on an educational level? - 1.9914460711655217
-----------------------------------------------------


# Averaging Word Embedding

In [10]:
import numpy as np

def load_glove_model(File):
    print("Loading Glove Model")
    glove_model = {}
    with open(File,'r') as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array(split_line[1:], dtype=np.float64)
            glove_model[word] = embedding
    print(f"{len(glove_model)} words loaded!")
    return glove_model

In [11]:
glove_embedding = load_glove_model('glove.6B.50d.txt')

Loading Glove Model
400000 words loaded!


In [12]:
glove_embedding['book']

array([-0.0076543,  0.93456  , -0.73189  , -0.55162  ,  0.76977  ,
        0.35925  , -1.1365   , -1.1632   ,  0.34214  ,  0.29145  ,
       -0.8711   ,  0.9197   , -0.47069  , -0.22834  ,  1.4777   ,
       -0.81714  , -0.17466  , -0.51093  , -0.28354  ,  0.23292  ,
        0.71832  ,  0.23414  ,  0.49443  ,  0.35483  ,  0.76889  ,
       -1.4374   , -1.7457   , -0.28994  , -0.10156  , -0.36959  ,
        2.5502   , -1.0581   , -0.049416 , -0.25524  , -0.63303  ,
        0.02671  , -0.18733  ,  0.20206  , -0.26288  , -0.41418  ,
        0.83473  , -0.14227  , -0.28125  ,  0.098155 , -0.17096  ,
        0.52408  ,  0.31851  , -0.089847 , -0.27223  , -0.0088736])

In [13]:
# function to calculate the average word based sentence embedding
def averaging(query) -> np.array:
    words_list = nltk.word_tokenize(query)
    length = 0 
    
    for word in words_list:
      if word in glove_embedding:
        length = length + 1         # calculate length of input query
    query_emb = []
    glove_index = []

    for word in words_list:    
      if word in glove_embedding: # check if word is present in glove embedding
        glove_index.append (glove_embedding[word])
    
    score = []
    if (length >= 1):             # if query has at least one word
      glove_index = np.array(glove_index)

      for i in range(0, 50):
        sum = 0
        for j in range(0, length ):
          elem = glove_index[j][i] # get value from word embedding
          sum = sum + elem

        score.append(sum)

      score = np.array(score)
    
      for elem in score:
        elem = float(elem/length) # divide by length of query to calculate average
        query_emb.append(elem)

      query_emb = np.array(query_emb)

    else:
      query_emb = np.zeros(100)

    return query_emb

In [14]:
import numpy
def neg_distance(pt1, pt2):
    sum_sq = np.sum(np.square(pt1 - pt2))
    return -np.sqrt(sum_sq)

In [15]:
# #function to caculate cosine similarity between two vectors v1 and v2
# def cosine(v1, v2):

#     v1 = np.array(v1)
#     v2 = np.array(v2)
    
#     denominator = (np.sqrt(np.sum(v1**2)) * np.sqrt(np.sum(v2**2)))

#     if  denominator != 0:
#       cos = float ( (np.dot(v1, v2)) / denominator )
#     else:
#       cos = -1.0 # lowest val of cos theata

#     return cos

In [16]:
vec = []

for ques in (df['processed_ques2'][0:5000]): 
  val = averaging(ques ) 
  vec.append(val)

# store word average sentence embedding
average_enbedding = np.array(vec) 
average_enbedding.shape 

<ipython-input-16-5e570034b2f4>:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  average_enbedding = np.array(vec)


(5000,)

In [17]:
# return top k document indices and scores
def get_similarity_avg_word(input, k) :

  input = text_preprocessing(input) # preprocess input text

  val_input = averaging(input)

  cosine_similarities = []
  doc_simi = {}

  i = 0
  for x in average_enbedding:
    try:
      sim_calculation = neg_distance(val_input,x)
    except:
      sim_calculation = -99
    cosine_similarities.append(sim_calculation)
    doc_simi[i] = sim_calculation 
    i = i + 1
  
  sorted_sim = sorted(doc_simi.items(), key = lambda kv:(kv[1], kv[0]),reverse = True ) # sort by value
  topk = sorted_sim[0:k]

  return   topk

In [18]:
def print_top_k(k: list):
  """
  function to print top k similar document
  """
  for i in k :
    val = i[0]
    score = i[1]
    sentence = df['question2'][val]
    print(" {}, Score: {}".format( sentence, score))

In [19]:
df['question1'][24]

'What are the best websites for entrepreneurs?'

In [20]:

top5 = get_similarity_avg_word("how to control my gut" , 5 )
print_top_k(top5)

 Since the nose has a breathing function, why does the mouth have a breathing function too? Is it redundant?, Score: -2.021339601787883
 How much time does the stomach take to send the first particles of chyme to the small intestine?, Score: -2.0919111153375307
 How do I reduce eye strain and headache while working on a computer?, Score: -2.134440984733282
 How do you slow down your digestive system?, Score: -2.1381674058986175
 If Einstein's theory of general relativity says that gravity is the result of curved spacetime and that matter, such as a planet, has no force whatsoever that pulls other matter towards it, how are we able to walk on the Earth without falling off? Is there a force pushing on us?, Score: -2.2206410086485895


In [21]:
def accuracy(df: pd.DataFrame, k):
    """
    function to perform inverted file TFIDF in for each row and return the accuracy
    :params:
        df: Test dataframe
        k: number of similar documents
    """
    tmp = 0
    for i in df.index: # for each row
        query = df['question1'][df.index == i].values[0]
        ques_sim = get_similarity_avg_word(query,k)
        for q in ques_sim: #for each similar question
            if q[0] == i:
                tmp += 1
                break
    return tmp/len(df)

In [22]:
accuracy_top2  = accuracy(df_test, 2) # top 2
accuracy_top5  = accuracy(df_test, 5) # top 5
 
print("Correctly predicted matches in top2: ", accuracy_top2)
print("Correctly predicted matches in top5: ", accuracy_top5) 

Correctly predicted matches in top2:  0.86
Correctly predicted matches in top5:  0.86


In [50]:
from AverageWordEmbedding import AverageWordEmbedding

In [51]:
model2 = AverageWordEmbedding('glove.6B.50d.txt', 50, df[:5000])

Loading Glove Model...
Embedding All Question 2 data...
Model is ready to query


/Users/MACOS/Documents/Learning/Adelaide/NLP/Text-Matching-System/AverageWordEmbedding.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self.q2_embedding = np.array(vec) # store average word embedding of all question 2


In [52]:
model2.accuracy(df_test, 5)

0.86

In [39]:
model2.query_engine()

Your query is: How to surf
 How do I train my cat to be a outdoor cat? - -3.504344720578614
-----------------------------------------------------
 Is it safe to swim in the ocean at night? - -3.804269284652978
-----------------------------------------------------
 Why does my Virtual DJ keeps crashing? - -3.866685477161496
-----------------------------------------------------
 How do websites like The Pirate Bay survive? - -3.9553926728751576
-----------------------------------------------------
 What is Paradise like? - -3.974708603592936
-----------------------------------------------------


In [62]:
# function to calculate the average word based sentence embedding
def averaging2(query) -> np.array:
    words_list = nltk.word_tokenize(query)
    length = 0 
    
    for word in words_list:
      if word in glove_embedding:
        length = length + 1         # calculate length of input query
    query_emb = []
    glove_index = []

    for word in words_list:    
      if word in glove_embedding: # check if word is present in glove embedding
        glove_index.append (glove_embedding[word])
    try:
        tmp = sum(glove_index)/length
    except:
        tmp = np.zeros(50)

    return tmp

In [63]:
arr1 = np.array([1,2,3,4,5])
arr2 = np.array([6,7,8,9,10])
(arr1 + arr2)/5

array([1.4, 1.8, 2.2, 2.6, 3. ])

In [64]:
averaging2('How to control my gut')

array([ 0.438865  , -0.26961825,  0.0690625 , -0.264885  ,  0.3105475 ,
        0.20371275,  0.1025725 ,  0.1583975 ,  0.236895  , -0.02950025,
        0.16521575,  0.3717595 , -0.43017   ,  0.0503195 ,  0.08789875,
        0.4204225 , -0.07227   , -0.13617275,  0.14628975, -0.385215  ,
       -0.128125  ,  0.265722  ,  0.2042175 , -0.21258835,  0.4479985 ,
       -1.7739625 , -0.4907435 ,  0.029605  ,  0.51749   , -0.3682475 ,
        3.047975  ,  0.5068575 , -0.5717475 , -0.03427   , -0.4175875 ,
        0.0467725 ,  0.054175  ,  0.24339525,  0.604275  , -0.21218   ,
       -0.01801825, -0.075164  , -0.1553185 ,  0.4495775 , -0.08939975,
        0.090298  ,  0.1472285 , -0.33631175,  0.030483  ,  0.0696875 ])

In [66]:
averaging('How to control my gut')

array([ 0.438865  , -0.26961825,  0.0690625 , -0.264885  ,  0.3105475 ,
        0.20371275,  0.1025725 ,  0.1583975 ,  0.236895  , -0.02950025,
        0.16521575,  0.3717595 , -0.43017   ,  0.0503195 ,  0.08789875,
        0.4204225 , -0.07227   , -0.13617275,  0.14628975, -0.385215  ,
       -0.128125  ,  0.265722  ,  0.2042175 , -0.21258835,  0.4479985 ,
       -1.7739625 , -0.4907435 ,  0.029605  ,  0.51749   , -0.3682475 ,
        3.047975  ,  0.5068575 , -0.5717475 , -0.03427   , -0.4175875 ,
        0.0467725 ,  0.054175  ,  0.24339525,  0.604275  , -0.21218   ,
       -0.01801825, -0.075164  , -0.1553185 ,  0.4495775 , -0.08939975,
        0.090298  ,  0.1472285 , -0.33631175,  0.030483  ,  0.0696875 ])